## Customer Sentiment Analysis


 For an IT company in technical support team , the employee has to  deal with different issues
from various customer in a daily basis. They’re using ticketing system and live chat to
communicate with customers. Customer feedback is very import to them – When They see positive
feedbacks, They know Their service is good and They may be able to expand their business with them.
When they see negative feedbacks,they need to take action to understand the reason and
address customer’s concerns timely.

So, They need to carefully monitor customer’s sentiment in all communications between their
support engineers and customers. Basically, They want to do at least two things 1) track
customer’s satisfaction level over times and give action suggestions; 2) real time monitor
ongoing communications and raise alarms when necessary.

However,  company’s data cannot be shared in public. So, this project will be using similar
data - “Customer Support on Twitter” dataset from Kaggle. And according to the effort and
limited time of this course, this project will focus on the first task “track customer’s satisfaction
level over times and give action suggestions”.



This notebook is being developed as Kaggle kernel using dataset https://www.kaggle.com/thoughtvector/customer-support-on-twitter and usig this kernel https://www.kaggle.com/soaxelbrooke/customer-sentiment-by-brand as reference.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:/Users/Rahul/Downloads/Customer Sentiment Analysis'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

C:/Users/Rahul/Downloads/Customer Sentiment Analysis\customer-sentiment-analysis.ipynb
C:/Users/Rahul/Downloads/Customer Sentiment Analysis\sample.csv
C:/Users/Rahul/Downloads/Customer Sentiment Analysis\twcs.csv
C:/Users/Rahul/Downloads/Customer Sentiment Analysis\twcs.csv.zip
C:/Users/Rahul/Downloads/Customer Sentiment Analysis\.ipynb_checkpoints\customer-sentiment-analysis-checkpoint.ipynb


In [2]:
# load libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm_notebook
from datetime import datetime

In [3]:
# Load customer feedback data from kaggle dataset
tweets = pd.read_csv('C:/Users/Rahul/Downloads/Customer Sentiment Analysis/twcs.csv')
tweets.shape

(2811774, 7)

In [5]:
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [4]:
tweets.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

Columns Meaning
- tweet_id

The unique ID for this tweet
- author_id

The unqiue ID for this tweet author (anonymized for non-company users)
- inbound

Whether or not the tweet was sent (inbound) to a company
- created_at

When the tweet was created
- text

The text content of the tweet

In [6]:
# We'll focus on what customers said to us, so let's get only customer messages
customer_msg = tweets[tweets.inbound]
customer_msg.shape

(1537843, 7)

In [7]:
customer_msg.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [8]:
# Original dataset includes support data from multiple companies. Here we choose one company with similar business as ours.
# for example Sprint.
customer_msg = customer_msg[customer_msg['text'].str.contains("sprintcare")]
customer_msg.shape

(13714, 7)

In [7]:
# Let's take a look at the data
customer_msg.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [9]:
# Reduce data size for better performance purpose in testing
customer_msg_sample = customer_msg.head(1000)
customer_msg_sample.shape

(1000, 7)

In [12]:
from tqdm.notebook import tqdm

tqdm.pandas()

# tqdm_notebook().pandas()

In [14]:
# Change data type to be datetime for column "created_at" and
# sort by the ascending order so we can analize the messages based on the time when they are created 
customer_msg_sample['created_at'] = pd.to_datetime(customer_msg_sample.created_at)
customer_msg_sample = customer_msg_sample.sort_values(by='created_at')

In [16]:
import nltk

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...


True

In [17]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Instantiate sentiment analyzer from NLTK, make "sentiment_analyze" function
sentiment_analyzer = SentimentIntensityAnalyzer()

def sentiment_analyze(text: str) -> float:
    return sentiment_analyzer.polarity_scores(text)['compound']


In [18]:
# # Instantiate sentiment analyzer from NLTK, make "sentiment_analyze" function
# sentiment_analyzer = SentimentIntensityAnalyzer()

# def sentiment_analyze(text: str) -> float:
#     return sentiment_analyzer.polarity_scores(text)['compound']


In [19]:
# Analyze customer sentiment based on their messages
customer_msg_sample['sentiment'] = \
    customer_msg_sample.text.progress_apply(sentiment_analyze)

In [20]:
customer_msg_sample.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,sentiment
149695,179663,158314,True,2017-08-02 21:12:17+00:00,@sprintcare manager at west Nyack sprint store...,179665,179662.0,0.5719
180942,214341,166909,True,2017-09-14 02:29:12+00:00,@sprintcare @166911 That makes two of us becau...,214340,214342.0,-0.3753
202517,237311,172516,True,2017-09-20 23:07:47+00:00,@115714 @sprintcare asked my wife to call the ...,237306,NaN,-0.3400
157561,189210,160394,True,2017-09-26 23:42:58+00:00,@sprintcare We need the sprint Wego app on iPh...,189209,NaN,0.4404
157559,189208,160394,True,2017-09-27 00:27:04+00:00,@sprintcare What does apple do in creating a s...,189207,189209.0,0.2960


In [21]:
# Group by author_id so we can deal with messages from different customers(author_id)
customer_grouped = customer_msg_sample.groupby('author_id')

In [22]:
# Case 1, we can calculate average sentiment values for each customer over this data chunk
# and compare to predefined alert threshold. 
# If the average sentiment value is lower than the threshold we'll list all the customers(author_id) 
# and suggest to reach to them for further discussion to find the reason and resolve the issues proactively
author_sentiment_avg = customer_grouped.sentiment.mean().sort_values()
author_sentiment_avg_df = pd.DataFrame({'author_id':author_sentiment_avg.index, 'sentiment':author_sentiment_avg.values})
alert_threshold_avg = -0.7
author_sentiment_avg_df[author_sentiment_avg_df.sentiment <= alert_threshold_avg]

,author_id,sentiment
0,157821,-0.9487
1,157432,-0.8722
2,128518,-0.8402
3,139643,-0.8264
4,136463,-0.8126
5,154080,-0.8020
6,126203,-0.8020
7,167686,-0.8011
8,160374,-0.7734
9,134203,-0.7698


In [23]:
# Case 2, we can find the minimum sentiment values for each customer over this data chunk
# and compare to predefined alert threshold. 
# If the minimum sentiment value is lower than the threshold we'll list all the customers(author_id) 
# and suggest to reach to them for further discussion to find the reason and resolve the issues proactively
author_sentiment_lowest = customer_grouped.sentiment.min().sort_values()
author_sentiment_lowest_df = pd.DataFrame({'author_id':author_sentiment_lowest.index, 'sentiment':author_sentiment_lowest.values})
alert_threshold_min = -0.8
author_sentiment_lowest_df[author_sentiment_lowest_df.sentiment <= alert_threshold_min]

,author_id,sentiment
0,157821,-0.9487
1,136460,-0.9301
2,150834,-0.9221
3,160379,-0.9211
4,173005,-0.8910
5,123402,-0.8778
6,157432,-0.8722
7,148370,-0.8689
8,166617,-0.8553
9,160370,-0.8514
